Install rarfile, We'll use it to parse the routes without decompressing the whole archive

In [ ]:
!pip install rarfile

Import modules and mount drive folder to work on colab

In [ ]:

import pandas as pd
from bs4 import BeautifulSoup
from os import listdir
from os.path import join
from datetime import datetime
import re
import rarfile
from os import mkdir, remove, listdir, walk
from os.path import join, exists

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Define the paths we are going to use

In [ ]:
line_ids = '337' 
uncompressed_archive_path = '/content/drive/MyDrive/Data_Mining/Project/ATAC_deflated/'
sqlite_path = '/content/drive/MyDrive/Data_Mining/Project/ATAC-Final.sqlite'

Define models for the sqlalchemy -> sqlite database mapping

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Boolean, ForeignKey, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import func
from sqlalchemy.orm import relationship

engine = create_engine('sqlite:///'+sqlite_path, echo=False)
Base = declarative_base()

# This represents one stop ( it is called routestop because multiple routes can share the same stop )
class RouteStop(Base):
  __tablename__ = 'routestops'
  route_id = Column(Integer, nullable=False, primary_key= True) # one route identifies a particular ordered sequence of stops
  stop_id = Column(Integer, nullable=False, primary_key= True)
  line_id = Column(String) # line id (the one on the top of the autobus)
  name = Column(String, nullable=False)
  position = Column(Integer, nullable=False, primary_key=True) # position in the route
  last = Column(Boolean, nullable=False, default=False) # TODO not implemented

  # buses = relationship("BusLocation", foreign_keys=[route_id, stop_id])

  def __str__(self):
    return f"[{self.position}@{self.route_id} | {self.name}({self.stop_id}) ]"
  
# One bus detected at a given spot at a given time
class BusLocation(Base):
  __tablename__ = 'buslocations'
  
  time = Column(DateTime, nullable=False, primary_key= True)
  bus_id = Column(Integer, nullable=False, primary_key= True)
  route_id = Column(Integer, ForeignKey('routestops.route_id'))
  stop_id = Column(Integer, ForeignKey('routestops.stop_id'))

  # stop = relationship("RouteStop", foreign_keys="[BusLocation.route_id, BusLocation.stop_id]")
  stop = relationship("RouteStop",
                    primaryjoin="and_(BusLocation.route_id==RouteStop.route_id, "
                        "BusLocation.stop_id==RouteStop.stop_id)")


  def __str__(self):
    return f"{self.time.hour}:{self.time.minute} [{self.bus_id}]  @  {self.stop_id} - {self.route_id} {self.stop}"

# State of a bus stop at a given time
class RouteStopState(Base):
   __tablename__ = 'routestopdelays'
   uuid = Column(Integer, primary_key=True, autoincrement=True)
   stop_id = Column(Integer, ForeignKey('routestops.stop_id'))
   route_id = Column(Integer, ForeignKey('routestops.route_id'))
   time = Column(DateTime, nullable=False)
   state = Column(Integer)
   
# Departure for a certain route at a certain time
class RouteDeparture(Base):
  __tablename__ = 'routedepartures'
  route_id = Column(Integer, ForeignKey('routestops.route_id'),primary_key= True)
  time = Column(DateTime, nullable=False, primary_key= True)
  day = Column(Date )
   

Base.metadata.create_all(engine)

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

The dataset is composed by a 7 gb rar  archive containing around 90 rar subarchives of 300MB of html pages each.
This code allows us to unarchive the files and parse the html pages in-memory without decompressing all the subarchives.


In [ ]:
# this parses the date of an html file to retrieve it's date 
def path2date(subarchive_path):
  try:
    m = re.search("(march|april|may|june|july)/[Ww]ebpage[s]?[ ]?[_]?([0-9]+)", subarchive_path)
    month_to_int = {'march':3,'april':4, 'may':5, 'june':6, 'july':7 }
    day = int(m.group(2))
    month = month_to_int[m.group(1)]
    year = 2020
  except:
    print(f"Error parsing {subarchive_path}")
    return None
  return date(day=day, month=month, year=year)



Various parsers for each model of the db

In [ ]:
import glob
from tqdm.notebook import tqdm
from datetime import date

def parse_routestop(line_id, route_id, i, soup_stop):
  name = soup_stop.text
  stop_id = int (re.search("/paline/palina/([0-9]+)[?]", soup_stop['href']).group(1) )
  new_stop = RouteStop(name=name,position=i, route_id=route_id, stop_id=stop_id, line_id=line_id  )
  return (new_stop)

def parse_buslocation(route_id, stop_id, download_time, soup_bus):
  bus_id = re.search("id_veicolo=([0-9]+)", soup_bus['href']).group(1)
  bl = BusLocation(route_id=route_id, time=download_time, stop_id=stop_id, bus_id=bus_id )

  return bl

def parse_routestopstate(route_id, stop_id, download_time, soup_line ):
  state_num = re.search('(-1|[0-5])', soup_line['class'][0]).group(1)[0]
  rds = RouteStopState(stop_id = stop_id, route_id = route_id, time = download_time, state= state_num)
  return rds
  

def parse_routedepartures(route_id, download_time, html_soup):
  soups = html_soup.select('span.b')
  deps = []
  for s in soups:
    try:
      hour = int(s.text[:-1])
    except:
      continue
    soup_minutes = s.nextSibling
    for m in str(soup_minutes).strip().split(' '):

        dep_time = datetime(2020, download_time.month, download_time.day, hour, int(m))
        deps.append( RouteDeparture(route_id = route_id, day = download_time.date(), time=dep_time))
  return deps






Main logic of the parser

In [ ]:


def parse_page(file_path, html, line_id, first_of_day=False, parsed_routes=[]):
  route_id =  file_path.split('/')[-2]
  filename =  file_path.split('/')[-1]
  download_time = datetime.strptime(filename.rsplit('.',1)[0], '%Y-%m-%d %H-%M-%S.%f')

  html_soup = BeautifulSoup(html)
  line_soups = html_soup.select( 'div[class^="stato"]')

  items = []
  to_add=None

  for e, soup_line in enumerate(line_soups):
    soup_links = soup_line.select('a')

    soup_stop = soup_links[-1]
    soup_bus = None
    if len(soup_links) > 1:
      soup_bus = soup_links[0]

    stop_item = parse_routestop(line_id, route_id, e, soup_stop)

    items.append(parse_routestopstate(route_id, stop_item.stop_id, download_time, soup_line ))
    if soup_bus:
      items.append(parse_buslocation(route_id, stop_item.stop_id, download_time, soup_bus ))

    if stop_item.route_id not in parsed_routes:
      items.append(stop_item)
      to_add=stop_item.route_id

  if to_add:
    parsed_routes += [to_add]

  if first_of_day:
    items += parse_routedepartures(route_id, download_time, html_soup)  

  session.add_all(items)

  return parsed_routes


def parse_pages(line_id, verbose=False):
  subarchive_paths = glob.glob(uncompressed_archive_path + "/**/*.rar", recursive = True)
  line_path_filter = f'/{line_id}/'
  parsed_routes = []
  for idx, subarchive_path in tqdm(enumerate(subarchive_paths)):
    archive_date = path2date(subarchive_path)
    first_of_day = True
    if session.query(BusLocation, RouteStop).filter(RouteStop.stop_id == BusLocation.stop_id, RouteStop.route_id == BusLocation.route_id).filter(BusLocation.time == archive_date).count() > 0 :
      print("Already imported")
      
      continue

    subarchive = rarfile.RarFile(subarchive_path)
    html_paths = subarchive.namelist()
    html_paths = [hp for hp in html_paths if ('.html' in hp) and (line_path_filter in hp)]
    for html_path in tqdm(html_paths, desc=f"Parsing line {line_id}, day {archive_date} "):
      
      html = subarchive.open(html_path).read()
      parsed_routes = parse_page(html_path, html, line_id,first_of_day, parsed_routes = parsed_routes )
      if first_of_day == True:
        first_of_day=False
        session.commit()
      

    session.commit()

Run the parser

In [ ]:
# Only try to parse numeric line_id if you have letters it may crash ( weird routes )
line_ids = ['337', '341', '60', '69', '64', '70', '88', '14'
session.rollback()
for line_id in line_ids :
  parse_pages(line_id, verbose=False)